In [297]:
import numpy as np
from numpy.linalg import*
import matplotlib.pyplot as plt
from numpy import r_, exp, cos, sin, pi, zeros, ones, hanning, sqrt, log, floor, reshape, mean
from scipy import signal
from numpy.fft import fft
import math
%matplotlib inline

In [316]:
# data import by readfile
data_path = './With_offset/IF_Impedance_BVD.csv' 
data = np.genfromtxt(data_path, delimiter=',')
Ns = len(data[1:,0])
f_data = data[1:,0]
real_data = data[1:,1]
imag_data = data[1:,2]

# Design parameters
R_S = 50   # Source resistance
R_SW = 5   # Switch ON resistance
N = 4      # Number of paths
n = 1      # reference vs target relative harmonic number difference  (ref - target)

f_S = 3.5  # [GHz]
f_LO = 1.9 # [GHz]  - after divider


In [266]:
print(int(100*(15+15)))
print(f_data)

3000
[-15.   -14.99 -14.98 ...  14.98  14.99  15.  ]


In [311]:
def Z_IF(freq):       # in [GHz]
    if abs(freq) > 15:
        return 0
    else:
        index = int(100*(freq+15))
        return real_data[index]+1j*imag_data[index]

def a_coef(x):
    return np.sinc(x/N)*exp(-1j*x*np.pi/N)/N

def G_sum(i,j):
    Sum = 0
    for k in range(-100,101):
        Sum += a_coef(N*k-n+i)*np.conjugate(a_coef(N*k-n+j))
    return N*Sum

def gamma(i):
    return N*a_coef(i)*np.conjugate(a_coef(i))
def R_sh(i):
    if i==0:
        return 1000000
    else:
        return (R_S+R_SW)/(1/(N*gamma(i))-1)

In [319]:
# Full Equation --function
def Zx_Full(f_S,f_LO, M):   # M: Number of harmonics to consider
  
    A = np.zeros((2*M+1,2*M+1),dtype='complex')
    b = np.zeros(2*M+1,dtype='complex')
    for i in range(-M,M+1):
        for j in range(-M,M+1):
            A[i+M,j+M] = Z_IF(f_S + (j-n)*f_LO)*G_sum(i,j)
        b[i+M] = a_coef(i-n)

    X_hat = np.matmul(np.linalg.inv((R_S+R_SW)*np.identity(2*M+1) + A),b)

    Harm_sum = 0
    for i in range(-M,M+1):
        Harm_sum += N*a_coef(n-i)*Z_IF(f_S+(i-n)*f_LO)*X_hat[i+M]

    return R_SW + (R_S+R_SW)/((1/Harm_sum)-1)

# LTI_Full sweep
NumSweep=31
f_start = 2 #GHz
df = 0.1  # GHz
Zx = np.zeros(NumSweep,dtype='complex')

for i in range(NumSweep):
    f_S = f_start + i*df
    f_LO = f_S - 1.6
    Zx[i]= Zx_Full(f_S,f_LO,10)

print(np.real(Zx))

[106.44604667 101.14364471 105.17667712 123.08834625 103.58474013
  95.69405102  87.72017641  87.51284351 103.07288485 126.52416605
 136.68752578 130.88056286 123.01605976 120.93385625 114.19389642
 108.39611514 104.93401582 100.96345605  98.44765218  98.04010123
  96.84943702  94.97720299  93.31025884  90.7893811   89.94419564
  88.58769145  87.21092238  85.39065863  82.96634716  86.94466529
  79.89172196]


In [317]:
# LTI model 1 -- function
def Zx_LTI_1(f_S,f_LO):
    M = 50   # Number of harmonics to consider
    Sum = 0
    for i in range(-M,M+1):
            Sum += gamma(i)*Z_IF(f_S + i*f_LO)*R_sh(i)/((gamma(i)*Z_IF(f_S + i*f_LO)) + R_sh(i))
    return R_SW + Sum

# LTI 1 sweep
NumSweep=31
f_start = 2 #GHz
df = 0.1  # GHz
Zx_LTI = np.zeros(NumSweep,dtype='complex')

for i in range(NumSweep):
    f_S = f_start + i*df
    f_LO = f_S - 1.6
    Zx_LTI[i]= Zx_LTI_1(f_S,f_LO)


In [320]:
with open('Zx_fullEQN.txt', 'w') as f:
    f.write('    '.join(str(i) for i in np.real(Zx))+'\n')
    f.write('    '.join(str(i) for i in np.imag(Zx)))

In [318]:
with open('Zx_LTI.txt', 'w') as f:
    f.write('    '.join(str(i) for i in np.real(Zx_LTI))+'\n')
    f.write('    '.join(str(i) for i in np.imag(Zx_LTI)))

In [270]:
# LTI model 1
M = 50   # Number of harmonics to consider
Harm_sum_LTI_1 = 0
for i in range(-M,M+1):
    Harm_sum_LTI_1 += gamma(i)*Z_IF(f_S + i*f_LO)*R_sh(i)/((gamma(i)*Z_IF(f_S + i*f_LO)) + R_sh(i))
Zx = R_SW + Harm_sum_LTI_1
print(Zx)

(15.516814589238754-1.03630940208921j)


In [283]:
# Full Equation

M = 10  # Number of harmonics to consider
A = np.zeros((2*M+1,2*M+1),dtype='complex')
b = np.zeros(2*M+1,dtype='complex')
for i in range(-M,M+1):
    for j in range(-M,M+1):
        A[i+M,j+M] = Z_IF(f_S + (j-n)*f_LO)*G_sum(i,j)
    b[i+M] = a_coef(i-n)

X_hat = np.matmul(np.linalg.inv((R_S+R_SW)*np.identity(2*M+1) + A),b)

Harm_sum = 0
for i in range(-M,M+1):
    Harm_sum += N*a_coef(n-i)*Z_IF(f_S+(i-n)*f_LO)*X_hat[i+M]

Z_x = R_SW + (R_S+R_SW)/((1/Harm_sum)-1)
print(Z_x)



(15.7210432202846-2.517594327559626j)


In [30]:
M=3
b = np.zeros(M+1)
print(b)

[0. 0. 0. 0.]


In [72]:
1/R_sh(0)

D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in cdouble_scalars
D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in cdouble_scalars
D:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in cdouble_scalars
  """Entry point for launching an IPython kernel.


(nan+nanj)